In [10]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

In [11]:
# Memory on LLMChain with PromptTemplate

from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, memory_key="chat_history"
)
template = """
You are a helpful AI talking to a human.

{chat_history}
Human: {question}
You:
"""
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="How are you?")
chain.predict(question="My name is Gianna")
chain.predict(question="What's my name?")
# memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human.


Human: How are you?
You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human.

Human: How are you?
AI: I'm an AI, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: My name is Gianna
You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human.

Human: How are you?
AI: I'm an AI, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: My name is Gianna
AI: Nice to meet you, Gianna! How can I assist you today?
Human: What's my name?
You:


> Finished chain.


'Your name is Gianna.'

In [12]:
# Memory on LLMChain with ChatPromptTemplate

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
chain = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

chain.predict(question="How are you?")
chain.predict(question="My name is Gianna")
chain.predict(question="What's my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: How are you?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: How are you?
AI: As an AI, I don't have feelings, but I'm here to assist you. How can I help you today?
Human: My name is Gianna

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: How are you?
AI: As an AI, I don't have feelings, but I'm here to assist you. How can I help you today?
Human: My name is Gianna
AI: Hello Gianna! How can I assist you today?
Human: What's my name?

> Finished chain.


'Your name is Gianna. Is there anything specific you would like assistance with, Gianna?'

In [18]:
# Memory on LCEL

from langchain.schema.runnable import RunnablePassthrough


def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


def invoke_chain(question):
    output = chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context({"input": question}, {"output": output.content})


invoke_chain("My name is Gianna")
invoke_chain("I like snacks")
invoke_chain("What do I like?")